## Math 157: Intro to Mathematical Software
## UC San Diego, Winter 2018

## Final project, part 2: Joshua Cho- Model Selection for Machine Learning

The following notebook will be using ideas presented in the Model Selection section on the scikit learn website. http://scikit-learn.org/stable/model_selection.html

### Model Selection: The Big Picture
Suppose you had a big midterm coming up and want to get a good score. Lucky for you, the professor has also provided a practice midterm to prepare you for the exam. You study only the material on the practice exam because you reason that the real exam will test topics similar to the practice exam. On exam day you recieve the midterm only to find out that it is the exact same test as the practice midterm. Having studied the practice midterm, you ace the test with a perfect score. However did the exam really test you on how much you learned? Real exams test you to see if you understand the concepts of the class by testing you on questions similar to the material, but most importantly, questions you have never seen before.

Similar to the midterm example above, model selection is a test to measure the predictive power on unforseen data. Like the practice midterm, if the machine studied the training data and was then tested on the same training data, it would ace the test, but that's not useful in the real world. Like students, the machine learns from the testing data and then predicts on some new data it has never seen before as a measure of how 'well' it has learned.

### Model Selection: Getting Started
For this presentation, I will be focusing on supervised Machine Learning. Supervised learning means that you have the labels of the training data whereas unsupervised learning means that you no have the labels for your training data. Following the midterm example, supervised learning would mean you are given the practice midterm (training data) and the solutions to each probelm (the desired labels). 

Since we are in the setting of supervised learning, our training data is made of 2 components: the many variables of the dataset and the corresponding labels. In this way, you can think of each model as a function that takes the inputs of the dataset and returns a label according to the function. We have X and Y, but we need to select the best function f. Since we are only given the training data and do not know what the real data will be, we split the training data into a training set and testing set. Then we procede to train our model on the smaller training set and predict on the testing set.

### Leave One Out Cross Validation (LOOCV)
We will start with leave-one-out because it is relatively simple. We have our dataset of $n$ samples: we have n pairs of labels and inputs. Now imagine that we did not have the 1st label. Then we could train a model on the remaining $n-1$ data points to predict the 1st label. Now imagine that we did not have the 2nd label. Then we could train a model on the remaining $n-1$ data points to predict the 2nd label. We can repeat the procedure $n$ times, each time pretending we do not have the label of the i-th data point and using the remaning $n-1$ data points to train a model and predict the missing label. Recall that in reality, we do have the value for the label we pretended to be 'missing' so we can compare the real label to the predicted label. We can then use the accuracy as a measure of the predictive power of our model.

Source: https://www.cs.cmu.edu/~schneide/tut5/node42.html

In [1]:
from sklearn import datasets
iris = datasets.load_iris()
iris.data.shape, iris.target.shape
#notice that we have 150 data points, each with 4 features and 150 labels

((150, 4), (150,))

In [2]:
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn import datasets
from sklearn import svm

#Here we perform LOOCV on the iris dataset, meaning we loop 150 times
LOOCV = LeaveOneOut()
scores = np.zeros(len(iris.data)); count = 0
for index_train, index_test in LOOCV.split(iris.data):
    trainX = iris.data[index_train]
    testX = iris.data[index_test]
    trainY = iris.target[index_train]
    testY = iris.target[index_test]

    #we train on the 149 data points and guess the missing label and see if the prediction is accurate or not
    clf = svm.SVC(kernel='linear', C=1).fit(trainX,trainY)
    scores[count] = clf.score(testX,testY)
    count+=1

#an array of accuracies
scores

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.])

Above we get 150 different estimators, each made from exclusing 1 point from the avaliable 150. We see an overwhelming majority of the estimators managed to predict the missing label correctly.

Notice that for large n, this method becomes very computationally infeasible. In addition, for large n, your training set that you make from the $n-1$ is indistinguishable from the entire set so it is like you are training on the entire dataset anyways. We need a smarter method that captures the idea of LOOCV but is computationally more efficient.

### Cross Validation
Since having n groups would be computationally infeasible, let's instead split the given data into 2 groups- with $p$ being the percentage of the given data that you want use for training. Then the size of the training set is $np$ and the size of the testing set is $n(1-p)$. Since we want our test model to be accurate, we want to feed it a larger partition of the data we have avalaible, thus we pick a $p\geq 0.5$

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm

#Splitting the avaliable data into a training set and testing set
p = .55
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=(1-p))
X_train.shape, y_train.shape

#notice that the training set is 150*.55

((82, 4), (82,))

In [4]:
#and the testing set is 150*(1-.55)
X_test.shape, y_test.shape

((68, 4), (68,))

In [7]:
#build a model on the training set, then pass the testing set into the constructed model and return it's accuracy score
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)

0.98529411764705888

Look! We get a high accuracy score! A job well done and we can stop right? Well, not quite. 

Since we randomly chose which data went into training and testing it could be the fact that we got quite lucky and we split the data nicely. This in turn might have cause the estimator to overfit- that is it learned so well from the training data that it does a bad job predicting new data it has never seen before.

### K-fold Cross Validation
So now we have learned that choosing n groups is too computationally expensive and choosing 2 groups can lead to overfitting. This leads us to K-fold Cross Validation which expands on the idea of LOOCV by forming $k$ groups as opposed to $n$ groups. Here $k$ is an integer which represents the number of groups you want to divide the dataset into, each with $\frac{n}{k}$ elements randomly assigned. Then you pretend the i-th group is your testing data and your train on the remaining $k-1$ groups and get the accuracy. You repeat the process k times and see which model returns the best score.

In [8]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

#Splitting the avaliable data into a k groups, then letting one group be the testing data and the remaining k-1 groups to be the training set. This procedure is repeated for each group as the testing data
p = .55
k = 5
clf = svm.SVC(kernel='linear', C=1)
folds = ShuffleSplit(n_splits = k, test_size = (1-p))
scores = cross_val_score(clf, iris.data, iris.target, cv = folds)
scores #returns k values

array([ 1.        ,  0.94117647,  0.91176471,  0.97058824,  0.98529412])

Since all the scores are similar, you can be sure that overfitting is not happening and that the estimator is doing a good job at predicitng the labels given this training dataset.

### Exercise 1: Iris Dataset
Below is a copy of the code above for testing the cross validation of the iris dataset. Pick a  $p \leq 0.5$. Explain why the score is still pretty good for $p \leq 0.5$. Hint: Do some research on the iris dataset

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm

p = .55
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=(1-p))
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)

0.97058823529411764

#Your reponse here

### Exercise 2: K-fold CV
Perform K-fold Cross Validation on the digits dataset below with $p=0.7$ and $k=10$ using the K-Nearest Neighbors estimator. Get the average score from the K-fold Cross Validation of the training set and see if score of the final testing data trained on the entire training set is reasonable.

In [10]:
from sklearn import datasets
digits = datasets.load_digits()
digits.data.shape, digits.target.shape

((1797, 64), (1797,))

In [11]:
dataX, final_testX, dataY, final_testY = train_test_split(digits.data, digits.target, test_size=.2)
dataX.shape, dataY.shape

((1437, 64), (1437,))

In [12]:
#do not forget to import the estimator
from sklearn.neighbors import KNeighborsRegressor

#K-Nearest Neighbors estimator using 9 of its neighbors
KNeighborsRegressor(n_neighbors=9)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=9, p=2,
          weights='uniform')

In [0]:
#Your code goes here


~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />
~ <br />

### Solutions

Exercise 1: Source: https://en.wikipedia.org/wiki/Iris_flower_data_set

The Iris dataset is a famous dataset because of how easily the 3 categories are sperated using most machine learning techniques. A $p \leq .5$ means that our training sample is smaller which normally is a bad choice due to the weak classifier built. However, since these 3 categories are so distinct, a classifier trained a fewer points is still able to predict the correct labels with high accuracy.

Exercise 2:

In [13]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
digits = datasets.load_digits()
digits.data.shape, digits.target.shape

dataX, final_testX, dataY, final_testY = train_test_split(digits.data, digits.target, test_size=.2)
dataX.shape, dataY.shape

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor #need to import

p = .7
k = 10
clf = KNeighborsRegressor(n_neighbors=9) #change the estimator
folds = ShuffleSplit(n_splits = k, test_size = (1-p))
scores = cross_val_score(clf, dataX, dataY, cv = folds)
print(mean(scores))

clf = KNeighborsRegressor(n_neighbors=9).fit(dataX,dataY)
clf.score(final_testX,final_testY)

0.938392014825


0.96406766511502406